https://s3-geospatial.s3-us-west-2.amazonaws.com/survey_results_public.csv

## 한 컬럼의 값이 여러개인 데이터 처리

- `split`
- `explode`

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set('spark.app.name', 'PySpark DataFrame 4')
conf.set('spark.master', 'local[*]')

spark = SparkSession.builder\
        .config(conf = conf)\
        .getOrCreate()

In [2]:
df = spark.read.csv('survey_results_public.csv', header = True)

df.show(5)

+----------+--------------------+--------------------+--------------------+--------+----------+--------------------+-------------+--------------------+---------+------------+--------------------+--------------------+-------------------+---------+--------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------------+----------------------------+-----------+--------------------+--------------------+--------------------+---------+--------------------+---------------+-------------+---------------+------+-----+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+-------------------+
|ResponseId|          MainBranch|          Employment|             Country|US_State|UK_Country|     

In [3]:
import pyspark.sql.functions as f

df = df.select(f.col('ResponseId'), f.col('LanguageHaveWorkedWith'), f.col('LanguageWantToWorkWith'))

df.show(5)

+----------+----------------------+----------------------+
|ResponseId|LanguageHaveWorkedWith|LanguageWantToWorkWith|
+----------+----------------------+----------------------+
|         1|  C++;HTML/CSS;Java...|                 Swift|
|         2|     JavaScript;Python|                    NA|
|         3|  Assembly;C;Python...|     Julia;Python;Rust|
|         4|  JavaScript;TypeSc...|  JavaScript;TypeSc...|
|         5|  Bash/Shell;HTML/C...|  Bash/Shell;HTML/C...|
+----------+----------------------+----------------------+
only showing top 5 rows



In [4]:
df_trim = df.withColumn('language_have', f.split(f.col('LanguageHaveWorkedWith'), ';'))

df_trim.show(5, truncate = False)

+----------+---------------------------------------------+------------------------------+----------------------------------------------------+
|ResponseId|LanguageHaveWorkedWith                       |LanguageWantToWorkWith        |language_have                                       |
+----------+---------------------------------------------+------------------------------+----------------------------------------------------+
|1         |C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift|Swift                         |[C++, HTML/CSS, JavaScript, Objective-C, PHP, Swift]|
|2         |JavaScript;Python                            |NA                            |[JavaScript, Python]                                |
|3         |Assembly;C;Python;R;Rust                     |Julia;Python;Rust             |[Assembly, C, Python, R, Rust]                      |
|4         |JavaScript;TypeScript                        |JavaScript;TypeScript         |[JavaScript, TypeScript]                            |

In [5]:
df_trim = df.withColumn('language_have', f.split(f.trim(f.col('LanguageHaveWorkedWith')), ';'))\
            .withColumn('language_want', f.split(f.trim(f.col('LanguageWantToWorkWith')), ';'))

df_trim.show(5, truncate = False)

+----------+---------------------------------------------+------------------------------+----------------------------------------------------+-----------------------------------+
|ResponseId|LanguageHaveWorkedWith                       |LanguageWantToWorkWith        |language_have                                       |language_want                      |
+----------+---------------------------------------------+------------------------------+----------------------------------------------------+-----------------------------------+
|1         |C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift|Swift                         |[C++, HTML/CSS, JavaScript, Objective-C, PHP, Swift]|[Swift]                            |
|2         |JavaScript;Python                            |NA                            |[JavaScript, Python]                                |[NA]                               |
|3         |Assembly;C;Python;R;Rust                     |Julia;Python;Rust             |[Assembly, C, Py

In [6]:
df_have = df_trim.select('ResponseId',
                         f.explode('language_have').alias('language_have'))

df_have.show(5)

+----------+-------------+
|ResponseId|language_have|
+----------+-------------+
|         1|          C++|
|         1|     HTML/CSS|
|         1|   JavaScript|
|         1|  Objective-C|
|         1|          PHP|
+----------+-------------+
only showing top 5 rows



In [7]:
df_want = df_trim.select('ResponseId',
                         f.explode('language_want').alias('language_want'))

df_want.show(5)

+----------+-------------+
|ResponseId|language_want|
+----------+-------------+
|         1|        Swift|
|         2|           NA|
|         3|        Julia|
|         3|       Python|
|         3|         Rust|
+----------+-------------+
only showing top 5 rows



### 정렬

SQL의 `ORDER BY`를 Spark로 해본다.

In [14]:
df_have_count = df_have.groupby('language_have').count()

df_have_count.show(10)

+-------------+-----+
|language_have|count|
+-------------+-----+
|           C#|22984|
|          VBA| 3847|
|         Rust| 5799|
|   Bash/Shell|22385|
|   JavaScript|53587|
|           NA| 1082|
|         Perl| 2028|
|       Erlang|  651|
|       Matlab| 3846|
|      Crystal|  466|
+-------------+-----+
only showing top 10 rows



In [16]:
df_have_count.sort('count').show(10)

+-------------+-----+
|language_have|count|
+-------------+-----+
|        COBOL|  437|
|      Crystal|  466|
|          APL|  536|
|       Erlang|  651|
|           F#|  804|
|        Julia| 1068|
|           NA| 1082|
|         LISP| 1096|
|       Elixir| 1438|
|      Clojure| 1552|
+-------------+-----+
only showing top 10 rows



In [17]:
df_have_count.sort(f.desc('count')).show(10)

+-------------+-----+
|language_have|count|
+-------------+-----+
|   JavaScript|53587|
|     HTML/CSS|46259|
|       Python|39792|
|          SQL|38835|
|         Java|29162|
|      Node.js|27975|
|   TypeScript|24909|
|           C#|22984|
|   Bash/Shell|22385|
|          C++|20057|
+-------------+-----+
only showing top 10 rows



In [19]:
df_have_count.orderBy('count').show(10)

+-------------+-----+
|language_have|count|
+-------------+-----+
|        COBOL|  437|
|      Crystal|  466|
|          APL|  536|
|       Erlang|  651|
|           F#|  804|
|        Julia| 1068|
|           NA| 1082|
|         LISP| 1096|
|       Elixir| 1438|
|      Clojure| 1552|
+-------------+-----+
only showing top 10 rows



In [20]:
df_have_count.orderBy('count', ascending = False).show(10)

+-------------+-----+
|language_have|count|
+-------------+-----+
|   JavaScript|53587|
|     HTML/CSS|46259|
|       Python|39792|
|          SQL|38835|
|         Java|29162|
|      Node.js|27975|
|   TypeScript|24909|
|           C#|22984|
|   Bash/Shell|22385|
|          C++|20057|
+-------------+-----+
only showing top 10 rows

